In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
import pandas as pd
from datetime import datetime
from IPython.core.display import HTML
HTML("<style>.container { width:100%; }</style>")

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

from dse_do_utils import DataManager, OptimizationEngine,ScenarioManager
from docplex.mp.model import Model

# Optimization Model Informations

In [3]:
MODEL_NAME = 'Unit_Commitment'
SCENARIO_NAME = 'Base_Scenario' 
sm = ScenarioManager(model_name=MODEL_NAME, scenario_name=SCENARIO_NAME, project = project)

mdl = Model(MODEL_NAME)

In [4]:
InputTables, outputTables = sm.load_data_from_scenario()
InputTables.keys()

dict_keys(['Units', 'Demand'])

In [5]:
Units = InputTables['Units']

display(Units.head(2))

Demand = InputTables['Demand']

display(Demand.head(2))

,index,energy,initial,min_gen,max_gen,operating_max_gen,min_uptime,min_downtime,ramp_up,ramp_down,start_cost,fixed_cost,variable_cost,Unit,co2_cost
0,0,coal,400,100.0,425,400,15,9,212.0,183.0,5000,208.61,22.536,coal1,30
1,1,coal,350,140.0,365,350,15,8,150.0,198.0,4550,117.37,31.985,coal2,30


,demand,Date,Date_Sequence
0,1259.0,2022-04-22,day1
1,1439.0,2022-04-23,day2


# Defining of Functions

Define a set of functions, which will be used later in the code.

In [6]:
def continuous_var_series(df, mdl,**kargs):
    return pd.Series(mdl.continuous_var_list(df.index, **kargs), index = df.index)

def binary_var_series(df, mdl,**kargs):
    return pd.Series(mdl.binary_var_list(df.index, **kargs), index = df.index)

def integer_var_series(df, mdl,**kargs):
    '''Create a Series of integer dvar for each row in the DF. Most effective method. Best practice.
    Result can be assigned to a column of the df.
    Usage:
        df['xDVar'] = mdl.integer_var_series(df, name = 'xDVar')
    Args:
        mdl: CPLEX Model
        df: DataFrame
        **kargs: arguments passed to mdl.integer_var_list method. E.g. 'name'
        
    :returns: pandas.Series with integer dvars, index matches index of df
    '''
    #We are re-using the index from the DF index:
    return pd.Series(mdl.integer_var_list(df.index, **kargs), index = df.index)


class CplexSum():
    """Function class that adds a series of dvars into a cplex sum expression.
    To be used as a custom aggregation in a groupby.
    Usage:
        df2 = df1.groupby(['a']).agg({'xDVar':CplexSum(engine.mdl)}).rename(columns={'xDVar':'expr'})

    Sums the dvars in the 'xDVar' column into an expression
    """
    def __init__(self, mdl):
        self.mdl = mdl
    def __call__(self, dvar_series):
        return self.mdl.sum(dvar_series)
    
    
def extract_solution(df, extract_dvar_names=None, drop_column_names=None, drop:bool=True):
    """Generalized routine to extract a solution value. 
    Can remove the dvar column from the df to be able to have a clean df for export into scenario."""
    if extract_dvar_names is not None:
        for xDVarName in extract_dvar_names:
            if xDVarName in df.columns:
                df[f'{xDVarName}_Solution'] = [dvar.solution_value for dvar in df[xDVarName]]
                if drop:
                    df = df.drop([xDVarName], axis = 1)
    if drop and drop_column_names is not None:
        for column in drop_column_names:
            if column in df.columns:
                df = df.drop([column], axis = 1)
    return df    

# Define decision variables

We define the decision variables based on each day and unit. First we create a table of all possible permutations

In [7]:
Unit_Date = pd.merge(Units[["Unit"]],Demand["Date_Sequence"], how = "cross" )

Unit_Date = Unit_Date.set_index(["Unit","Date_Sequence"], verify_integrity = True)

Unit_Date.sample(5)

Empty DataFrame
Columns: []
Index: [(gas2, day120), (diesel2, day73), (gas3, day160), (gas3, day116), (gas1, day63)]

The $x_{InUse} (i,t)$ is true if the unit $i$ is in production at period $t$

In [8]:
Unit_Date["x_InUse"] = binary_var_series(Unit_Date, mdl,name = 'x_InUse')

The $x_{TurnOn} (i,t)$  true if unit $i$ is turned on at period $t$

In [9]:
Unit_Date["x_TurnOn"] = binary_var_series(Unit_Date, mdl,name = 'x_TurnOn')

$x_{TurnOff} (i,t)$ is true if unit $i$ is switched off at period $t$

In [10]:
Unit_Date["x_TurnOff"] = binary_var_series(Unit_Date, mdl,name = 'x_TurnOff')                #continuous_var_series

The $x_{Production} (i,t)$  production of energy for unit $i$ at period $t$

In [11]:
Unit_Date["x_Production"] = continuous_var_series(Unit_Date, mdl,name = 'x_Production')

Let's store all decision variablesion in a dataframe

In [12]:
Unit_Date = Unit_Date.reset_index(drop = False)

In [13]:
Unit_Date.head()

,Unit,Date_Sequence,x_InUse,x_TurnOn,x_TurnOff,x_Production
0,coal1,day1,x_InUse_coal1_day1,x_TurnOn_coal1_day1,x_TurnOff_coal1_day1,x_Production_coal1_day1
1,coal1,day2,x_InUse_coal1_day2,x_TurnOn_coal1_day2,x_TurnOff_coal1_day2,x_Production_coal1_day2
2,coal1,day3,x_InUse_coal1_day3,x_TurnOn_coal1_day3,x_TurnOff_coal1_day3,x_Production_coal1_day3
3,coal1,day4,x_InUse_coal1_day4,x_TurnOn_coal1_day4,x_TurnOff_coal1_day4,x_Production_coal1_day4
4,coal1,day5,x_InUse_coal1_day5,x_TurnOn_coal1_day5,x_TurnOff_coal1_day5,x_Production_coal1_day5


In [14]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 7680
   - binary=5760, integer=0, continuous=1920
 - number of constraints: 0
   - linear=0
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Constraints

### 1- Linking in-use status to production
Whenever the unit is in use, the production must be within the minimum and maximum generation.


$x_{Production} (i,t)  <= x_{TurnOn} (i,t) \times MaxGen(i) $

$x_{Production} (i,t)  >= x_{TurnOn} (i,t) \times MinGen(i) $

In [15]:
Production_range = pd.merge ( Unit_Date[["Unit","x_InUse","x_Production"]] , Units[["Unit","min_gen","max_gen"]], on = "Unit")

Production_range.head()

,Unit,x_InUse,x_Production,min_gen,max_gen
0,coal1,x_InUse_coal1_day1,x_Production_coal1_day1,100.0,425
1,coal1,x_InUse_coal1_day2,x_Production_coal1_day2,100.0,425
2,coal1,x_InUse_coal1_day3,x_Production_coal1_day3,100.0,425
3,coal1,x_InUse_coal1_day4,x_Production_coal1_day4,100.0,425
4,coal1,x_InUse_coal1_day5,x_Production_coal1_day5,100.0,425


When in use, the production level is constrained to be between min and max generation.

In [16]:
for row in Production_range.itertuples():
    mdl.add_constraint ( row.x_Production <= row.x_InUse * row.max_gen )
    mdl.add_constraint ( row.x_Production >= row.x_InUse * row.min_gen )

### Initial state

If initial production is nonzero, then period $t=1$ is not a turn_on. Else $x_{TurnOn} (i,t)=0 $ 

Dual logic is implemented for turn_off

In [17]:
initial_state  = Units [["Unit","initial"]]
day_1 = Unit_Date[Unit_Date["Date_Sequence"]=="day1"]

initial_state = pd.merge (initial_state, day_1, on = "Unit")
initial_state

,Unit,initial,Date_Sequence,x_InUse,x_TurnOn,x_TurnOff,x_Production
0,coal1,400,day1,x_InUse_coal1_day1,x_TurnOn_coal1_day1,x_TurnOff_coal1_day1,x_Production_coal1_day1
1,coal2,350,day1,x_InUse_coal2_day1,x_TurnOn_coal2_day1,x_TurnOff_coal2_day1,x_Production_coal2_day1
2,gas1,205,day1,x_InUse_gas1_day1,x_TurnOn_gas1_day1,x_TurnOff_gas1_day1,x_Production_gas1_day1
3,gas2,52,day1,x_InUse_gas2_day1,x_TurnOn_gas2_day1,x_TurnOff_gas2_day1,x_Production_gas2_day1
4,gas3,155,day1,x_InUse_gas3_day1,x_TurnOn_gas3_day1,x_TurnOff_gas3_day1,x_Production_gas3_day1
5,gas4,150,day1,x_InUse_gas4_day1,x_TurnOn_gas4_day1,x_TurnOff_gas4_day1,x_Production_gas4_day1
6,diesel1,78,day1,x_InUse_diesel1_day1,x_TurnOn_diesel1_day1,x_TurnOff_diesel1_day1,x_Production_diesel1_day1
7,diesel2,76,day1,x_InUse_diesel2_day1,x_TurnOn_diesel2_day1,x_TurnOff_diesel2_day1,x_Production_diesel2_day1
8,diesel3,0,day1,x_InUse_diesel3_day1,x_TurnOn_diesel3_day1,x_TurnOff_diesel3_day1,x_Production_diesel3_day1
9,diesel4,0,day1,x_InUse_diesel4_day1,x_TurnOn_diesel4_day1,x_TurnOff_diesel4_day1,x_Production_diesel4_day1


In [18]:
for row in initial_state.itertuples():
    if row.initial > 0 :   
            
            mdl.add_constraint  ( row.x_TurnOn == 0)                   #if u is already running, not starting up
            
            mdl.add_constraint  ( row.x_TurnOff +  row.x_InUse == 1)   # turnoff iff not in use
            
            
    else: 
        
           mdl.add_constraint ( row.x_TurnOn  == row.x_InUse )        # turn on at 1 iff in use at 1
            
           mdl.add_constraint(row.x_TurnOff ==0)                      # already off, not switched off at t==1

In [19]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 7680
   - binary=5760, integer=0, continuous=1920
 - number of constraints: 3860
   - linear=3860
 - parameters: defaults
 - objective: none
 - problem type is: MILP


###  Ramp-up / ramp-down constraint

ramp up and down constraints for day 1
    
Variations of the production level over time in a unit is constrained by a ramp-up / ramp-down process.


In [20]:
for unit in Units["Unit"].values:
    a = Unit_Date [ (Unit_Date["Unit"]==unit) & (Unit_Date["Date_Sequence"]=="day1")]
    b = Units [ Units["Unit"]==unit][["Unit","initial","ramp_up","ramp_down"]]
    r = pd.merge (a , b,  on ="Unit")
    
    mdl.add_constraint ( r.x_Production[0] - r.initial[0] <= r.ramp_up[0] )
    
    mdl.add_constraint( r.initial[0] - r.x_Production[0] <= r.ramp_down[0])
    

ramp up and down constraints for remaining days   
    
We use the  pandas groupby operation to collect all decision variables for each unit in separate series. Then, we iterate over units to post constraints enforcing the ramp-up / ramp-down process by setting upper bounds on the variation of the production level for consecutive periods.

$x_{Production} (i,t+1) - x_{Production} (i,t) <= {RampUp}(i)$

$x_{Production} (i,t) - x_{Production} (i,t+1) >= {RampDown}(i)$

In [21]:
Ramp_Table = pd.DataFrame()

for unit in Units["Unit"].values:
    a = Unit_Date[Unit_Date["Unit"]==unit][["Unit","Date_Sequence","x_Production"]]
    a.rename(columns = {"x_Production":"Current_Day_Production"}, inplace = True)   
    a["Next_Day_Production"] = a["Current_Day_Production"].shift(-1)       # shift 1 day to calculate next day production
    a = a.iloc[0:-1]                                                       # get rid of last row, which has a NaN for next day production
    
    Ramp_Table = pd.concat([Ramp_Table,a])

Ramp_Table = pd.merge(Ramp_Table,Units[["Unit",'ramp_up','ramp_down']], on = "Unit") 

Ramp_Table.sample(7)

,Unit,Date_Sequence,Current_Day_Production,Next_Day_Production,ramp_up,ramp_down
1395,diesel2,day59,x_Production_diesel2_day59,x_Production_diesel2_day60,60.0,45.0
913,gas3,day150,x_Production_gas3_day150,x_Production_gas3_day151,58.0,77.5
1896,diesel4,day178,x_Production_diesel4_day178,x_Production_diesel4_day179,12.0,12.0
1747,diesel4,day29,x_Production_diesel4_day29,x_Production_diesel4_day30,12.0,12.0
1481,diesel2,day145,x_Production_diesel2_day145,x_Production_diesel2_day146,60.0,45.0
1110,gas4,day156,x_Production_gas4_day156,x_Production_gas4_day157,50.0,60.0
1442,diesel2,day106,x_Production_diesel2_day106,x_Production_diesel2_day107,60.0,45.0


In [22]:
for row in Ramp_Table.itertuples():
    mdl.add_constraint(row.Next_Day_Production - row.Current_Day_Production <= row.ramp_up)
    
    mdl.add_constraint(row.Current_Day_Production - row.Next_Day_Production <= row.ramp_down)

In [23]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 7680
   - binary=5760, integer=0, continuous=1920
 - number of constraints: 7700
   - linear=7700
 - parameters: defaults
 - objective: none
 - problem type is: MILP


###  Turn on / turn off
The following constraints determine when a unit is turned on or off.


if unit is off at time $t$ and on at time $t+1$, then it was turned on at time $t+1$

if unit is on at time $t$ and off at time $t+1$, then it was turned off at time $t+1$

In [24]:
Status = pd.DataFrame([])

for unit in Units["Unit"].values:
    a = Unit_Date[Unit_Date["Unit"]==unit][["Unit","x_InUse","x_TurnOn","x_TurnOff"]]
    
    a.rename(columns = {"x_InUse":"Current_Day_InUse"}, inplace = True)
    a.rename(columns = {"x_TurnOn":"Current_Day_TurnOn"}, inplace = True)
    a.rename(columns = {"x_TurnOff":"Current_Day_TurnOff"}, inplace = True)
    
    a["Next_Day_InUse"] = a["Current_Day_InUse"].shift(-1)       # shift 1 day to calculate next day InUse
    a["Next_Day_TurnOn"] = a["Current_Day_TurnOn"].shift(-1)       # shift 1 day to calculate next day TurnOn
    a["Next_Day_TurnOff"] = a["Current_Day_TurnOff"].shift(-1)       # shift 1 day to calculate next day TurnOff
    
    a = a.iloc[0:-1]                                                # get rid of last row, which has a NaN
    
    Status = pd.concat([Status,a])
        
display(Status.sample(5))

,Unit,Current_Day_InUse,Current_Day_TurnOn,Current_Day_TurnOff,Next_Day_InUse,Next_Day_TurnOn,Next_Day_TurnOff
951,gas3,x_InUse_gas3_day184,x_TurnOn_gas3_day184,x_TurnOff_gas3_day184,x_InUse_gas3_day185,x_TurnOn_gas3_day185,x_TurnOff_gas3_day185
791,gas3,x_InUse_gas3_day24,x_TurnOn_gas3_day24,x_TurnOff_gas3_day24,x_InUse_gas3_day25,x_TurnOn_gas3_day25,x_TurnOff_gas3_day25
184,coal1,x_InUse_coal1_day185,x_TurnOn_coal1_day185,x_TurnOff_coal1_day185,x_InUse_coal1_day186,x_TurnOn_coal1_day186,x_TurnOff_coal1_day186
1176,diesel1,x_InUse_diesel1_day25,x_TurnOn_diesel1_day25,x_TurnOff_diesel1_day25,x_InUse_diesel1_day26,x_TurnOn_diesel1_day26,x_TurnOff_diesel1_day26
1805,diesel4,x_InUse_diesel4_day78,x_TurnOn_diesel4_day78,x_TurnOff_diesel4_day78,x_InUse_diesel4_day79,x_TurnOn_diesel4_day79,x_TurnOff_diesel4_day79


In [25]:
for row in Status.itertuples():
    
    #print(row.Next_Day_InUse - row.Current_Day_InUse <= row.Next_Day_TurnOn)
    mdl.add_constraint(row.Next_Day_InUse - row.Current_Day_InUse <= row.Next_Day_TurnOn)
    
    #print(row.Current_Day_InUse - row.Next_Day_InUse + row.Next_Day_TurnOn == row.Next_Day_TurnOff)
    mdl.add_constraint(row.Current_Day_InUse - row.Next_Day_InUse + row.Next_Day_TurnOn == row.Next_Day_TurnOff)

In [26]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 7680
   - binary=5760, integer=0, continuous=1920
 - number of constraints: 11520
   - linear=11520
 - parameters: defaults
 - objective: none
 - problem type is: MILP


### Minimum uptime and downtime
When a unit is turned on, it cannot be turned off before a minimum uptime. Conversely, when a unit is turned off, it cannot be turned on again before a minimum downtime.


$\Large\sum_{t \in \tau} x_{TurnOn} (i,t) <=  x_{InUse} (i,t)   \qquad   {for} \qquad   \tau = \{t- MinUptime, ..., t+1\}$

$\Large\sum_{t \in \tau} x_{TurnOff} (i,t) <=  1- x_{InUse} (i,t)   \qquad   {for} \qquad   \tau = \{t- MinDowntime, ..., t+1\}$

In [27]:
T = len(Demand)

for unit in Units["Unit"].values:
    a= Units[Units["Unit"]==unit][["Unit","min_uptime","min_downtime"]]
    
    min_uptime = a.min_uptime.values[0] 
    min_downtime = a.min_downtime.values[0] 
    
    for t in range(min_uptime,T):
        mdl.add_constraint(mdl.sum(  Unit_Date[(t - min_uptime) + 1   :   t + 1 ][["x_TurnOn"]].values) <= Unit_Date["x_InUse"][t] ) 
        
        
    for t in range(min_downtime, T):   
        mdl.add_constraint(mdl.sum( Unit_Date[(t - min_downtime) + 1:t + 1]["x_TurnOff"].values)  <= 1 - Unit_Date["x_InUse"][t]  ) 

In [28]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 7680
   - binary=5760, integer=0, continuous=1920
 - number of constraints: 15263
   - linear=15263
 - parameters: defaults
 - objective: none
 - problem type is: MILP


### Demand

Total production level must be equal or higher than demand on any period.

This time, the pandas operation groupby is performed on "periods" since we have to iterate over the list of all units for each period.

$\Large\sum_{i \in AllUnits} x_{TurnOn} (i,t) <=  P_{demand}(t)$


In [29]:
Total_day_Production = Unit_Date[["Date_Sequence","x_Production"]].groupby("Date_Sequence").agg(CplexSum(mdl)).rename(columns={"x_Production":"Total_day_Production"})
Total_day_Production = pd.merge(Total_day_Production, Demand, on = "Date_Sequence")

Total_day_Production.head(2)

,Date_Sequence,Total_day_Production,demand,Date
0,day1,x_Production_coal1_day1+x_Production_coal2_day1+x_Production_gas1_day1+x_Production_gas2_day1+x_Production_gas3_day1+x_Production_gas4_day1+x_Production_diesel1_day1+x_Production_diesel2_day1+x_Production_diesel3_day1+x_Production_diesel4_day1,1259.0,2022-04-22
1,day10,x_Production_coal1_day10+x_Production_coal2_day10+x_Production_gas1_day10+x_Production_gas2_day10+x_Production_gas3_day10+x_Production_gas4_day10+x_Production_diesel1_day10+x_Production_diesel2_day10+x_Production_diesel3_day10+x_Production_diesel4_day10,1158.0,2022-05-01


In [30]:
for row in Total_day_Production.itertuples():
    
    mdl.add_constraint(row.Total_day_Production >= row.demand)

In [31]:
mdl.print_information()

Model: Unit_Commitment
 - number of variables: 7680
   - binary=5760, integer=0, continuous=1920
 - number of constraints: 15455
   - linear=15455
 - parameters: defaults
 - objective: none
 - problem type is: MILP


# Define KPIs

In [32]:
cost = pd.merge ( Unit_Date, Units[["Unit","fixed_cost","variable_cost","start_cost","co2_cost"]], on = "Unit")
cost.head()

,Unit,Date_Sequence,x_InUse,x_TurnOn,x_TurnOff,x_Production,fixed_cost,variable_cost,start_cost,co2_cost
0,coal1,day1,x_InUse_coal1_day1,x_TurnOn_coal1_day1,x_TurnOff_coal1_day1,x_Production_coal1_day1,208.61,22.536,5000,30
1,coal1,day2,x_InUse_coal1_day2,x_TurnOn_coal1_day2,x_TurnOff_coal1_day2,x_Production_coal1_day2,208.61,22.536,5000,30
2,coal1,day3,x_InUse_coal1_day3,x_TurnOn_coal1_day3,x_TurnOff_coal1_day3,x_Production_coal1_day3,208.61,22.536,5000,30
3,coal1,day4,x_InUse_coal1_day4,x_TurnOn_coal1_day4,x_TurnOff_coal1_day4,x_Production_coal1_day4,208.61,22.536,5000,30
4,coal1,day5,x_InUse_coal1_day5,x_TurnOn_coal1_day5,x_TurnOff_coal1_day5,x_Production_coal1_day5,208.61,22.536,5000,30


$ FixedCost = \Large\sum_{t} \sum_{i} x_{TurnOn} (i,t)  \times FixedCost(i)$

$ VariableCost = \Large\sum_{t} \sum_{i} x_{Production} (i,t)  \times VariableCost(i)$

$ StartupCost = \Large\sum_{t} \sum_{i} x_{TurnOn} (i,t)  \times StartCost(i)$

$ CO2Cost = \Large\sum_{t} \sum_{i} x_{Production} (i,t)  \times CO2Cost(i)$

$ InUseDays = \Large\sum_{t} \sum_{i} x_{InUse} (i,t)  $

$ StartingTimes = \Large\sum_{t} \sum_{i} x_{TurnOn} (i,t)  $

In [33]:
total_fixed_cost = mdl.sum(cost.x_InUse * cost.fixed_cost)
total_variable_cost = mdl.sum(cost.x_Production * cost.variable_cost)
total_startup_cost = mdl.sum(cost.x_TurnOn * cost.start_cost)
total_co2_cost = mdl.sum(cost.x_Production * cost.co2_cost)

total_economic_cost = total_fixed_cost + total_variable_cost + total_startup_cost

total_nb_used = mdl.sum(Unit_Date.x_InUse)
total_nb_starts = mdl.sum(Unit_Date.x_TurnOn)


# store expression kpis to retrieve them later.
mdl.add_kpi(total_fixed_cost   , "Total Fixed Cost")
mdl.add_kpi(total_variable_cost, "Total Variable Cost")
mdl.add_kpi(total_startup_cost , "Total Startup Cost")
mdl.add_kpi(total_economic_cost, "Total Economic Cost")
mdl.add_kpi(total_co2_cost     , "Total CO2 Cost")
mdl.add_kpi(total_nb_used, "Total #used")
mdl.add_kpi(total_nb_starts, "Total #starts")

DecisionKPI(name=Total #starts,expr=x_TurnOn_coal1_day1+x_TurnOn_coal1_day2+x_TurnOn_coal1_day3+x_Tu..)

### Define Opjective Function

$\Large 𝐹𝑖𝑥𝑒𝑑𝐶𝑜𝑠𝑡 + 𝑉𝑎𝑟𝑖𝑎𝑏𝑙𝑒𝐶𝑜𝑠𝑡+  𝑆𝑡𝑎𝑟𝑡𝑢𝑝𝐶𝑜𝑠𝑡 + 𝐶𝑂2𝐶𝑜𝑠𝑡$

In [34]:
# minimize sum of all costs
mdl.minimize(total_fixed_cost + total_variable_cost + total_startup_cost + total_co2_cost)

## Solve

In [35]:
mdl.solve()
mdl.report()

* model Unit_Commitment solved with objective = 14209787.418
*  KPI: Total Fixed Cost    = 157934.630
*  KPI: Total Variable Cost = 8831892.288
*  KPI: Total Startup Cost  = 13439.000
*  KPI: Total Economic Cost = 9003265.918
*  KPI: Total CO2 Cost      = 5206521.500
*  KPI: Total #used         = 1318.000
*  KPI: Total #starts       = 12.000


### Create output tables

In [36]:
Unit_Date.head(2)

,Unit,Date_Sequence,x_InUse,x_TurnOn,x_TurnOff,x_Production
0,coal1,day1,x_InUse_coal1_day1,x_TurnOn_coal1_day1,x_TurnOff_coal1_day1,x_Production_coal1_day1
1,coal1,day2,x_InUse_coal1_day2,x_TurnOn_coal1_day2,x_TurnOff_coal1_day2,x_Production_coal1_day2


In [37]:
plan = extract_solution(Unit_Date,extract_dvar_names= ['x_TurnOn','x_TurnOff','x_Production','x_InUse'] , drop=True)
plan = pd.merge(plan, Demand[["Date","Date_Sequence"]], on = "Date_Sequence")
plan.round(2).head()

,Unit,Date_Sequence,x_TurnOn_Solution,x_TurnOff_Solution,x_Production_Solution,x_InUse_Solution,Date
0,coal1,day1,0.0,0.0,425.0,1.0,2022-04-22
1,coal2,day1,0.0,0.0,215.0,1.0,2022-04-22
2,gas1,day1,0.0,0.0,109.4,1.0,2022-04-22
3,gas2,day1,0.0,0.0,52.0,1.0,2022-04-22
4,gas3,day1,0.0,0.0,165.0,1.0,2022-04-22


## postprocessing 

Creating extra output table for visualization

In [38]:
Power_portions = plan[["Unit","Date_Sequence","x_Production_Solution"]].groupby(["Date_Sequence","Unit"]).sum().round().reset_index(drop = False)
Power_portions = pd.merge(Power_portions, Demand[["Date","Date_Sequence"]], on = "Date_Sequence")
del Power_portions["Date_Sequence"]
Power_portions.head()

,Unit,x_Production_Solution,Date
0,coal1,425.0,2022-04-22
1,coal2,215.0,2022-04-22
2,diesel1,90.0,2022-04-22
3,diesel2,87.0,2022-04-22
4,diesel3,0.0,2022-04-22


In [39]:
plant_percentage = plan[["Unit","Date_Sequence","x_Production_Solution"]].groupby(["Unit"]).sum().round().reset_index(drop = False)

plant_percentage["percentage"] = round (plant_percentage["x_Production_Solution"]/plant_percentage["x_Production_Solution"].sum(), 2)

plant_percentage.head()

,Unit,x_Production_Solution,percentage
0,coal1,81600.0,0.32
1,coal2,62345.0,0.25
2,diesel1,17280.0,0.07
3,diesel2,16704.0,0.07
4,diesel3,0.0,0.00


## Store the outputs into DO experiment, excel or csv

In [40]:
OutputTables = {}

OutputTables['plan'] = plan
OutputTables['Power_portions'] = Power_portions
OutputTables['plant_percentage'] = plant_percentage



for i in InputTables.keys():
    project.save_data(file_name = i+".csv", data = InputTables[i].to_csv(), overwrite=True)

for i in OutputTables.keys():
    project.save_data(file_name = i+".csv", data = OutputTables[i].to_csv(), overwrite=True)

In [43]:
sm.outputs = OutputTables
sm.update_solve_output_into_scenario(mdl, OutputTables)

sm.write_data_to_excel("UC_IO")

'/home/wsuser/work/UC_IO.xlsx'

### Export to  algebraic representation (LP)

https://www.ibm.com/docs/en/icos/12.8.0.0?topic=cplex-lp-file-format-algebraic-representation

In [45]:
sm.export_model_as_lp(mdl)

'/home/wsuser/work/Unit_Commitment.lp'